In [1]:
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np

pd.options.display.float_format = "{:.2f}".format
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
%matplotlib inline

import featuretools as ft

In [2]:
train = pd.read_csv('/home/dissertation/data/train_v2.csv', 
                          dtype = {'msno' : 'category'})

members = pd.read_csv('/home/dissertation/data/members_v3.csv',
                            dtype={'registered_via' : np.uint8,
                                   'gender' : str,
                                   'city' : 'category',
                                   'registered_via' : 'category'})

members['registration_init_time_dt'] = pd.to_datetime(members['registration_init_time'], 
                                                            format='%Y%m%d', errors='ignore')

## Next load in the transactions data
transactions = pd.read_csv('/home/dissertation/data/transactions.csv',
                                 dtype = {'payment_method' : 'category',
                                          'payment_plan_days' : np.uint8,
                                          'plan_list_price' : np.uint8,
                                          'actual_amount_paid': np.uint8,
                                          'is_auto_renew' : np.bool,
                                          'is_cancel' : np.bool})


In [6]:
merged_input = pd.merge(left=train, right=members, how='inner', on=['msno'])
merged_input.drop(['is_churn'], axis=1, inplace=True)
merged_input.head()

,msno,city,bd,gender,registered_via,registration_init_time,registration_init_time_dt
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,5,28,male,3,20131223,2013-12-23
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,13,20,male,3,20131223,2013-12-23
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13,18,male,3,20131227,2013-12-27
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,0,NaN,7,20140109,2014-01-09
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13,35,female,7,20140125,2014-01-25


In [7]:
import featuretools.variable_types as vtypes

# Make empty entityset
es = ft.EntitySet(id = 'customers')

In [8]:
# Create entity from members
es.entity_from_dataframe(entity_id='members', dataframe=merged_input,
                         index = 'msno', time_index = 'registration_init_time', 
                         variable_types = {'city': vtypes.Categorical, 
                                           'registered_via': vtypes.Categorical})

Entityset: customers
  Entities:
    members [Rows: 860967, Columns: 7]
  Relationships:
    No relationships

In [9]:
# Difference between listing price and price paid
transactions['price_difference'] = transactions['plan_list_price'] - transactions['actual_amount_paid']

# Planned price per day
transactions['planned_daily_price'] = transactions['plan_list_price'] / transactions['payment_plan_days']

# Actual price per day
transactions['daily_price'] = transactions['actual_amount_paid'] / transactions['payment_plan_days']

In [10]:
# Create entity from transactions
es.entity_from_dataframe(entity_id='transactions', dataframe=transactions,
                         index = 'transactions_index', make_index = True,
                         time_index = 'transaction_date', 
                         variable_types = {'payment_method_id': vtypes.Categorical, 
                                           'is_auto_renew': vtypes.Boolean, 'is_cancel': vtypes.Boolean})

Entityset: customers
  Entities:
    members [Rows: 860967, Columns: 7]
    transactions [Rows: 21547746, Columns: 13]
  Relationships:
    No relationships

In [11]:
# Relationships (parent, child)
r_member_transactions = ft.Relationship(es['members']['msno'], es['transactions']['msno'])

es.add_relationships([r_member_transactions])

Entityset: customers
  Entities:
    members [Rows: 860967, Columns: 7]
    transactions [Rows: 21547746, Columns: 13]
  Relationships:
    transactions.msno -> members.msno

In [35]:
feature_defs = ft.dfs(entityset=es, target_entity='members', 
                      where_primitives = ['sum', 'mean'],
                      max_depth=2, features_only=True)

In [36]:
print(f'This will generate {len(feature_defs)} features.')

This will generate 62 features.


In [32]:
import random; random.seed(42)

random.sample(feature_defs, 10)

[<Feature: MIN(transactions.transaction_date)>,
 <Feature: SUM(transactions.actual_amount_paid)>,
 <Feature: gender>,
 <Feature: MEAN(transactions.actual_amount_paid)>,
 <Feature: STD(transactions.membership_expire_date)>,
 <Feature: STD(transactions.actual_amount_paid)>,
 <Feature: STD(transactions.plan_list_price)>,
 <Feature: SUM(transactions.transaction_date)>,
 <Feature: DAY(registration_init_time_dt)>,
 <Feature: SUM(transactions.plan_list_price)>]

In [15]:
all_p = ft.list_primitives()
trans_p = all_p.loc[all_p['type'] == 'transform'].copy()
agg_p = all_p.loc[all_p['type'] == 'aggregation'].copy()

pd.options.display.max_colwidth = 100
agg_p.head()

,name,type,description
0,median,aggregation,Determines the middlemost number in a list of values.
1,n_most_common,aggregation,Determines the `n` most common elements.
2,num_true,aggregation,Counts the number of `True` values.
3,mean,aggregation,Computes the average for a list of values.
4,max,aggregation,"Calculates the highest value, ignoring `NaN` values."


In [16]:
# Specify aggregation primitives
agg_primitives = ['sum', 'time_since_last', 'avg_time_between', 'all', 'mode', 'num_unique', 'min', 'last', 
                  'mean', 'percent_true', 'max', 'std', 'count']

In [21]:
# Specify transformation primitives
trans_primitives = ['cum_sum', 'day', 'month', 'diff', 'time_since_previous']

In [19]:
# Specify where primitives
where_primitives = ['sum', 'mean', 'percent_true', 'all', 'any']

In [23]:
feature_defs = ft.dfs(entityset=es, target_entity='members', 
                      agg_primitives = agg_primitives,
                      trans_primitives = trans_primitives,
                      where_primitives = where_primitives,
                      cutoff_time_in_index = True,
                      max_depth = 2, features_only = True)

In [37]:
print(f'This will generate {len(feature_defs)} features.')

This will generate 62 features.


In [33]:
random.sample(feature_defs, 15)

[<Feature: MIN(transactions.planned_daily_price)>,
 <Feature: MEAN(transactions.actual_amount_paid)>,
 <Feature: MODE(transactions.payment_method_id)>,
 <Feature: SKEW(transactions.price_difference)>,
 <Feature: SUM(transactions.payment_plan_days)>,
 <Feature: MIN(transactions.payment_plan_days)>,
 <Feature: MAX(transactions.planned_daily_price)>,
 <Feature: registration_init_time>,
 <Feature: gender>,
 <Feature: YEAR(registration_init_time_dt)>,
 <Feature: STD(transactions.payment_plan_days)>,
 <Feature: STD(transactions.plan_list_price)>,
 <Feature: SKEW(transactions.transaction_date)>,
 <Feature: MIN(transactions.plan_list_price)>,
 <Feature: COUNT(transactions)>]

In [38]:
from timeit import default_timer as timer

start = timer()
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity='members', 
#                                       agg_primitives = agg_primitives,
#                                       trans_primitives = trans_primitives,
#                                       where_primitives = where_primitives,
                                      where_primitives = ['sum', 'mean'],
                                      max_depth = 2, features_only = False,
                                      verbose = 1, chunk_size = 1000,  
                                      n_jobs = 1,
                                      cutoff_time_in_index = True)
end = timer()
print(f'{round(end - start)} seconds elapsed.')

Built 62 features



Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/861 chunks


Elapsed: 00:05 | Remaining: 1:13:14 | Progress:   0%|          | Calculated: 1/861 chunks


Elapsed: 00:08 | Remaining: 1:04:50 | Progress:   0%|          | Calculated: 2/861 chunks


Elapsed: 00:11 | Remaining: 58:56 | Progress:   0%|          | Calculated: 3/861 chunks  


Elapsed: 00:14 | Remaining: 54:43 | Progress:   0%|          | Calculated: 4/861 chunks


Elapsed: 00:17 | Remaining: 51:46 | Progress:   1%|          | Calculated: 5/861 chunks


Elapsed: 00:20 | Remaining: 49:39 | Progress:   1%|          | Calculated: 6/861 chunks


Elapsed: 00:24 | Remaining: 48:09 | Progress:   1%|          | Calculated: 7/861 chunks


Elapsed: 00:27 | Remaining: 47:03 | Progress:   1%|          | Calculated: 8/861 chunks


Elapsed: 00:30 | Remaining: 46:20 | Progress:   1%|          | Calculated: 9/861 chunks


Elapsed: 00:33 | Remaining: 45:50 | Progress:   1%|          | Calculated: 10

Elapsed: 04:20 | Remaining: 41:53 | Progress:   9%|▉         | Calculated: 81/861 chunks


Elapsed: 04:23 | Remaining: 41:39 | Progress:  10%|▉         | Calculated: 82/861 chunks


Elapsed: 04:26 | Remaining: 41:31 | Progress:  10%|▉         | Calculated: 83/861 chunks


Elapsed: 04:29 | Remaining: 41:17 | Progress:  10%|▉         | Calculated: 84/861 chunks


Elapsed: 04:33 | Remaining: 41:18 | Progress:  10%|▉         | Calculated: 85/861 chunks


Elapsed: 04:36 | Remaining: 41:13 | Progress:  10%|▉         | Calculated: 86/861 chunks


Elapsed: 04:39 | Remaining: 41:07 | Progress:  10%|█         | Calculated: 87/861 chunks


Elapsed: 04:42 | Remaining: 40:54 | Progress:  10%|█         | Calculated: 88/861 chunks


Elapsed: 04:45 | Remaining: 40:59 | Progress:  10%|█         | Calculated: 89/861 chunks


Elapsed: 04:48 | Remaining: 40:58 | Progress:  10%|█         | Calculated: 90/861 chunks


Elapsed: 04:52 | Remaining: 40:49 | Progress:  11%|█         | Calculated: 91/861 chunks



Elapsed: 08:39 | Remaining: 37:27 | Progress:  19%|█▉        | Calculated: 162/861 chunks


Elapsed: 08:42 | Remaining: 37:25 | Progress:  19%|█▉        | Calculated: 163/861 chunks


Elapsed: 08:45 | Remaining: 37:21 | Progress:  19%|█▉        | Calculated: 164/861 chunks


Elapsed: 08:48 | Remaining: 37:23 | Progress:  19%|█▉        | Calculated: 165/861 chunks


Elapsed: 08:51 | Remaining: 37:20 | Progress:  19%|█▉        | Calculated: 166/861 chunks


Elapsed: 08:55 | Remaining: 37:12 | Progress:  19%|█▉        | Calculated: 167/861 chunks


Elapsed: 08:58 | Remaining: 37:02 | Progress:  20%|█▉        | Calculated: 168/861 chunks


Elapsed: 09:01 | Remaining: 36:57 | Progress:  20%|█▉        | Calculated: 169/861 chunks


Elapsed: 09:04 | Remaining: 36:58 | Progress:  20%|█▉        | Calculated: 170/861 chunks


Elapsed: 09:07 | Remaining: 36:56 | Progress:  20%|█▉        | Calculated: 171/861 chunks


Elapsed: 09:11 | Remaining: 37:03 | Progress:  20%|█▉        | Calculated: 172/8

Elapsed: 13:02 | Remaining: 33:23 | Progress:  28%|██▊       | Calculated: 243/861 chunks


Elapsed: 13:06 | Remaining: 33:16 | Progress:  28%|██▊       | Calculated: 244/861 chunks


Elapsed: 13:09 | Remaining: 33:17 | Progress:  28%|██▊       | Calculated: 245/861 chunks


Elapsed: 13:12 | Remaining: 33:12 | Progress:  29%|██▊       | Calculated: 246/861 chunks


Elapsed: 13:15 | Remaining: 33:05 | Progress:  29%|██▊       | Calculated: 247/861 chunks


Elapsed: 13:18 | Remaining: 32:56 | Progress:  29%|██▉       | Calculated: 248/861 chunks


Elapsed: 13:22 | Remaining: 32:57 | Progress:  29%|██▉       | Calculated: 249/861 chunks


Elapsed: 13:25 | Remaining: 32:57 | Progress:  29%|██▉       | Calculated: 250/861 chunks


Elapsed: 13:28 | Remaining: 32:54 | Progress:  29%|██▉       | Calculated: 251/861 chunks


Elapsed: 13:31 | Remaining: 32:48 | Progress:  29%|██▉       | Calculated: 252/861 chunks


Elapsed: 13:35 | Remaining: 32:49 | Progress:  29%|██▉       | Calculated: 253/8

Elapsed: 17:26 | Remaining: 29:10 | Progress:  38%|███▊      | Calculated: 324/861 chunks


Elapsed: 17:29 | Remaining: 28:59 | Progress:  38%|███▊      | Calculated: 325/861 chunks


Elapsed: 17:32 | Remaining: 29:02 | Progress:  38%|███▊      | Calculated: 326/861 chunks


Elapsed: 17:35 | Remaining: 28:59 | Progress:  38%|███▊      | Calculated: 327/861 chunks


Elapsed: 17:39 | Remaining: 28:58 | Progress:  38%|███▊      | Calculated: 328/861 chunks


Elapsed: 17:42 | Remaining: 28:58 | Progress:  38%|███▊      | Calculated: 329/861 chunks


Elapsed: 17:45 | Remaining: 28:57 | Progress:  38%|███▊      | Calculated: 330/861 chunks


Elapsed: 17:48 | Remaining: 28:49 | Progress:  38%|███▊      | Calculated: 331/861 chunks


Elapsed: 17:52 | Remaining: 28:45 | Progress:  39%|███▊      | Calculated: 332/861 chunks


Elapsed: 17:55 | Remaining: 28:42 | Progress:  39%|███▊      | Calculated: 333/861 chunks


Elapsed: 17:58 | Remaining: 28:43 | Progress:  39%|███▉      | Calculated: 334/8

...

Elapsed: 26:16 | Remaining: 20:30 | Progress:  56%|█████▋    | Calculated: 486/861 chunks


Elapsed: 26:19 | Remaining: 20:28 | Progress:  57%|█████▋    | Calculated: 487/861 chunks


Elapsed: 26:22 | Remaining: 20:25 | Progress:  57%|█████▋    | Calculated: 488/861 chunks


Elapsed: 26:25 | Remaining: 20:20 | Progress:  57%|█████▋    | Calculated: 489/861 chunks


Elapsed: 26:29 | Remaining: 20:18 | Progress:  57%|█████▋    | Calculated: 490/861 chunks


Elapsed: 26:32 | Remaining: 20:10 | Progress:  57%|█████▋    | Calculated: 491/861 chunks


Elapsed: 26:35 | Remaining: 20:19 | Progress:  57%|█████▋    | Calculated: 492/861 chunks


Elapsed: 26:39 | Remaining: 20:12 | Progress:  57%|█████▋    | Calculated: 493/861 chunks


Elapsed: 26:42 | Remaining: 20:06 | Progress:  57%|█████▋    | Calculated: 494/861 chunks


Elapsed: 26:45 | Remaining: 20:00 | Progress:  57%|█████▋    | Calculated: 495/861 chunks


Elapsed: 26:48 | Remaining: 20:01 | Progress:  58%|█████▊    | Calculated: 496/8

Elapsed: 30:43 | Remaining: 16:58 | Progress:  66%|██████▌   | Calculated: 567/861 chunks


Elapsed: 30:46 | Remaining: 17:23 | Progress:  66%|██████▌   | Calculated: 568/861 chunks


Elapsed: 30:50 | Remaining: 17:06 | Progress:  66%|██████▌   | Calculated: 569/861 chunks


Elapsed: 30:53 | Remaining: 16:53 | Progress:  66%|██████▌   | Calculated: 570/861 chunks


Elapsed: 30:57 | Remaining: 16:41 | Progress:  66%|██████▋   | Calculated: 571/861 chunks


Elapsed: 31:00 | Remaining: 16:41 | Progress:  66%|██████▋   | Calculated: 572/861 chunks


Elapsed: 31:03 | Remaining: 16:32 | Progress:  67%|██████▋   | Calculated: 573/861 chunks


Elapsed: 31:07 | Remaining: 16:27 | Progress:  67%|██████▋   | Calculated: 574/861 chunks


Elapsed: 31:10 | Remaining: 16:30 | Progress:  67%|██████▋   | Calculated: 575/861 chunks


Elapsed: 31:14 | Remaining: 16:12 | Progress:  67%|██████▋   | Calculated: 576/861 chunks


Elapsed: 31:17 | Remaining: 16:01 | Progress:  67%|██████▋   | Calculated: 577/8

Elapsed: 35:25 | Remaining: 12:48 | Progress:  75%|███████▌  | Calculated: 648/861 chunks


Elapsed: 35:28 | Remaining: 12:40 | Progress:  75%|███████▌  | Calculated: 649/861 chunks


Elapsed: 35:32 | Remaining: 12:33 | Progress:  75%|███████▌  | Calculated: 650/861 chunks


Elapsed: 35:36 | Remaining: 12:29 | Progress:  76%|███████▌  | Calculated: 651/861 chunks


Elapsed: 35:39 | Remaining: 12:23 | Progress:  76%|███████▌  | Calculated: 652/861 chunks


Elapsed: 35:43 | Remaining: 12:20 | Progress:  76%|███████▌  | Calculated: 653/861 chunks


Elapsed: 35:46 | Remaining: 12:13 | Progress:  76%|███████▌  | Calculated: 654/861 chunks


Elapsed: 35:50 | Remaining: 12:09 | Progress:  76%|███████▌  | Calculated: 655/861 chunks


Elapsed: 35:53 | Remaining: 12:06 | Progress:  76%|███████▌  | Calculated: 656/861 chunks


Elapsed: 35:57 | Remaining: 12:07 | Progress:  76%|███████▋  | Calculated: 657/861 chunks


Elapsed: 36:01 | Remaining: 12:09 | Progress:  76%|███████▋  | Calculated: 658/8

Elapsed: 40:20 | Remaining: 08:26 | Progress:  85%|████████▍ | Calculated: 729/861 chunks


Elapsed: 40:24 | Remaining: 08:14 | Progress:  85%|████████▍ | Calculated: 730/861 chunks


Elapsed: 40:28 | Remaining: 08:21 | Progress:  85%|████████▍ | Calculated: 731/861 chunks


Elapsed: 40:31 | Remaining: 08:10 | Progress:  85%|████████▌ | Calculated: 732/861 chunks


Elapsed: 40:35 | Remaining: 08:05 | Progress:  85%|████████▌ | Calculated: 733/861 chunks


Elapsed: 40:40 | Remaining: 08:24 | Progress:  85%|████████▌ | Calculated: 734/861 chunks


Elapsed: 40:44 | Remaining: 08:35 | Progress:  85%|████████▌ | Calculated: 735/861 chunks


Elapsed: 40:50 | Remaining: 09:34 | Progress:  85%|████████▌ | Calculated: 736/861 chunks


Elapsed: 40:55 | Remaining: 10:04 | Progress:  86%|████████▌ | Calculated: 737/861 chunks


Elapsed: 41:00 | Remaining: 09:44 | Progress:  86%|████████▌ | Calculated: 738/861 chunks


Elapsed: 41:03 | Remaining: 09:03 | Progress:  86%|████████▌ | Calculated: 739/8

Elapsed: 45:58 | Remaining: 02:51 | Progress:  94%|█████████▍| Calculated: 810/861 chunks


Elapsed: 46:01 | Remaining: 02:47 | Progress:  94%|█████████▍| Calculated: 811/861 chunks


Elapsed: 46:05 | Remaining: 02:44 | Progress:  94%|█████████▍| Calculated: 812/861 chunks


Elapsed: 46:08 | Remaining: 02:40 | Progress:  94%|█████████▍| Calculated: 813/861 chunks


Elapsed: 46:11 | Remaining: 02:37 | Progress:  95%|█████████▍| Calculated: 814/861 chunks


Elapsed: 46:15 | Remaining: 02:33 | Progress:  95%|█████████▍| Calculated: 815/861 chunks


Elapsed: 46:18 | Remaining: 02:31 | Progress:  95%|█████████▍| Calculated: 816/861 chunks


Elapsed: 46:21 | Remaining: 02:27 | Progress:  95%|█████████▍| Calculated: 817/861 chunks


Elapsed: 46:25 | Remaining: 02:24 | Progress:  95%|█████████▌| Calculated: 818/861 chunks


Elapsed: 46:28 | Remaining: 02:20 | Progress:  95%|█████████▌| Calculated: 819/861 chunks


Elapsed: 46:31 | Remaining: 02:18 | Progress:  95%|█████████▌| Calculated: 820/8

In [40]:
feature_matrix.columns

Index(['bd', 'gender', 'registration_init_time', 'city', 'registered_via',
       'SUM(transactions.payment_plan_days)',
       'SUM(transactions.plan_list_price)',
       'SUM(transactions.actual_amount_paid)',
       'SUM(transactions.transaction_date)',
       'SUM(transactions.membership_expire_date)',
       'SUM(transactions.price_difference)',
       'SUM(transactions.planned_daily_price)',
       'SUM(transactions.daily_price)', 'STD(transactions.payment_plan_days)',
       'STD(transactions.plan_list_price)',
       'STD(transactions.actual_amount_paid)',
       'STD(transactions.transaction_date)',
       'STD(transactions.membership_expire_date)',
       'STD(transactions.price_difference)',
       'STD(transactions.planned_daily_price)',
       'STD(transactions.daily_price)', 'MAX(transactions.payment_plan_days)',
       'MAX(transactions.plan_list_price)',
       'MAX(transactions.actual_amount_paid)',
       'MAX(transactions.transaction_date)',
       'MAX(transactions.

In [42]:
feature_matrix.head()

,,bd,gender,registration_init_time,city,registered_via,SUM(transactions.payment_plan_days),SUM(transactions.plan_list_price),SUM(transactions.actual_amount_paid),SUM(transactions.transaction_date),SUM(transactions.membership_expire_date),...,MEAN(transactions.daily_price),COUNT(transactions),PERCENT_TRUE(transactions.is_auto_renew),PERCENT_TRUE(transactions.is_cancel),NUM_UNIQUE(transactions.payment_method_id),MODE(transactions.payment_method_id),DAY(registration_init_time_dt),YEAR(registration_init_time_dt),MONTH(registration_init_time_dt),WEEKDAY(registration_init_time_dt)
msno,time,,,,,,,,,,,,,,,,,,,,,
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,inf,0,NaN,20140608,1,7,120.00,396.00,396.00,80662661.00,80671860.00,...,3.30,4.00,1.00,0.00,1.00,41.00,8,2014,6,6
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,inf,26,male,20101118,15,9,543.00,2682.00,2831.00,382953223.00,382983061.00,...,inf,19.00,1.00,0.00,1.00,39.00,18,2010,11,3
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,inf,0,NaN,20140927,1,7,750.00,3725.00,3874.00,524076576.00,524096776.00,...,inf,26.00,1.00,0.00,1.00,41.00,27,2014,9,5
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,inf,21,male,20160315,18,7,360.00,1788.00,1788.00,241947980.00,241957980.00,...,4.97,12.00,1.00,0.00,1.00,41.00,15,2016,3,1
++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,inf,32,male,20120122,22,9,543.00,2682.00,2831.00,382953223.00,382983137.00,...,inf,19.00,1.00,0.00,1.00,39.00,22,2012,1,6
